**Задание 1**

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [236]:
import pandas as pd
import time

In [154]:
df = pd.read_csv('ratings_2.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


**Вариант решения 1** Посчитано среднее время для всех пользователей, которые поставили более 100 оценок.

In [247]:
users = df.groupby('userId').agg({'timestamp': ['min', 'max', 'count']}).reset_index()
users_filtred = users.timestamp[users.timestamp['count'] > 100]
users_live = users_filtred['max'] - users_filtred['min']
avg = round((sum(users_live) / len(users_live)) / 86400) #В 1 дне 1 86400 секунд
print(f'Cреднее время жизни пользователей, которые выставили более 100 оценок {avg} дня.')

Cреднее время жизни пользователей, которые выставили более 100 оценок 464 дня


**Вариант решения 2** Посчитано время жизни в днях для каждого пользователя, который поставил более 100 оценок.

In [303]:
users_live_2 = round((users_filtred['max'] - users_filtred['min']) / 86400).reset_index()
users_live_2.columns=['user_Id', 'live_user']
users_live_2.head()

,user_Id,live_user
0,3,2.0
1,7,1.0
2,14,5456.0
3,16,0.0
4,18,0.0


**Вариант решение 3** Посчитано время жизни в секундах для каждого пользователя, который поставил более 100 оценок.

In [302]:
users_live_3 = (users_filtred['max'] - users_filtred['min']).reset_index()
users_live_3.columns=['user_Id', 'live_user']
users_live_3.head()

,user_Id,live_user
0,3,203560
1,7,85187
2,14,471393496
3,16,8053
4,18,5282


**Задание  2**

Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов. 

Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.


In [248]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [249]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [250]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [251]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [258]:
merge_table = rzd.merge(auto, how='outer').merge(air, how='outer')
merge_table.fillna(0.0, inplace=True)
merge_table

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [260]:
merge_table_2 = merge_table.merge(client_base, how='left')
merge_table_2

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4


**Задание 3**

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.



**Ответ**

1. Вопрос немного не понятен, но , предположим нам поступили данные с user_id и координатами и мы хотим эти данные добавить в таблицу визитов и покупок. Эти данные можно склетить по айдишнику юзера.
2. В итоге получим таблицу :

In [307]:
users_visit = pd.DataFrame(
    {
        'user_id': [1, 2],
        'latitude': [40.25, 54.97],
        'longitude': [25.38, 44.66],
        'visits': [15, 27],
        'purchases': [2, 4]
    }
)
users_visit

,user_id,latitude,longitude,visits,purchases
0,1,40.25,25.38,15,2
1,2,54.97,44.66,27,4


3. По итогу можно предположить, что наше склеивание не имеет смысла, так как мы и так могли видеть сколько было покупок и визитов без координат, но, если координаты перевести в конкретные места на карте, прикинуть что там расположено, а еще сравнить ползователей и сделать группировку по метам откуда наибольшее частое посещение и покупки можно получиь интересные инсайты)))

p.s. в случае большого разброса координат процесс идентификации места значитьльно усложнится)

В заключение отмечу, что описание представил на основании своих рассуждений и мыслей (так как нет четких данных, сложно понять что конкретно требуется). Допускаю что возможно задание было чуть иное))) 